# Short-selling

In [1]:
import numpy as np
import pandas as pd
pd.options.display.float_format = '{:.2f}'.format
import plotly.graph_objects as go
from scipy.stats import norm

### Margin with short positions

In [2]:
# Initial margin
INIT_VALUE = 6000
EQUITY = 3000

margin = EQUITY
short_proceeds  = INIT_VALUE

pct_margin = EQUITY / INIT_VALUE
print(f'Initial percent margin is: {pct_margin: .1%}')

Initial percent margin is:  50.0%


In [3]:
# Percent margin after price increase (ignoring interest income accumulation)
RETURN = 70/60-1

new_asset_value = INIT_VALUE*(1+RETURN)
new_equity = short_proceeds + margin - new_asset_value
pct_margin = (new_equity)/new_asset_value
print(f'New percent margin is: {pct_margin: .1%}')

New percent margin is:  28.6%


In [4]:
# Percent margin after price increase (ignoring interest income accumulation)
RETURN = 70/60-1

new_asset_value = INIT_VALUE*(1+RETURN)
new_equity = short_proceeds + margin - new_asset_value
pct_margin = (new_equity)/new_asset_value
print(f'New percent margin is: {pct_margin: .1%}')

New percent margin is:  28.6%


In [5]:
def short_margin_call_return(init_value, init_margin, mm):
    ''' 
    init_value: initial market value of short position
    init_margin: initial additional margin above short proceeds
    mm: maintenance margin percentage (in decimal notation)
    '''
    init_assets = init_value + init_margin
    return init_assets/(init_value*(1+mm)) -1 

ret = short_margin_call_return(INIT_VALUE, margin, 0.30)
print(f'Margin call occurs if return is at least: {ret: .1%}')


Margin call occurs if return is at least:  15.4%


### Simulated position (ignoring interest income and lending fees)

In [6]:
# Simulate levered account
MAINTENANCE_MARGIN = 0.35
T = 20

# set up account
cols = ['short_proceeds', 'margin', 'short_position', 'equity', 'ret', 'pct_margin', 'margin call?']
indx = np.arange(0,T)
acct = pd.DataFrame(dtype=float,columns=cols, index=indx)

# Initialize accounts
acct.loc[0,'short_position'] = INIT_VALUE
acct.loc[0,'equity']         = EQUITY
acct['short_proceeds'] = INIT_VALUE         # ignore accumulation of interest income less lending fee
acct['margin']         = margin             # ignore accumulation of interest income

# simulate T returns (NOTE: remove random_state to make this truly random)
acct.ret = norm.rvs(loc=0.05, scale = 0.1, size=T, random_state=26)  
# acct.ret = norm.rvs(loc=0.04, scale = 0.2, size=T)  

# Calculate asset and equity values + percent margin each period
for t in acct.index[1:]:
    acct.loc[t,'short_position'] = acct.loc[t-1,'short_position'] * (1+acct.loc[t,'ret'])
acct.equity = acct.short_proceeds + acct.margin - acct.short_position
acct.pct_margin = acct.equity/acct.short_position

# Margin calls occur when pct_margin drops below maintenance margin
acct['margin call?'] = np.where(acct.pct_margin < MAINTENANCE_MARGIN, 1, 0)
acct

,short_proceeds,margin,short_position,equity,ret,pct_margin,margin call?
0,6000,3000,6000.00,3000.00,0.07,0.50,0
1,6000,3000,5135.26,3864.74,-0.14,0.75,0
2,6000,3000,5758.57,3241.43,0.12,0.56,0
3,6000,3000,6427.80,2572.20,0.12,0.40,0
4,6000,3000,6602.08,2397.92,0.03,0.36,0
5,6000,3000,7111.60,1888.40,0.08,0.27,1
6,6000,3000,7446.50,1553.50,0.05,0.21,1
7,6000,3000,6508.56,2491.44,-0.13,0.38,0
8,6000,3000,6317.02,2682.98,-0.03,0.42,0
9,6000,3000,7026.19,1973.81,0.11,0.28,1


In [7]:
# Plot return time-series
fig = go.Figure()
trace= go.Scatter(x=acct.index, y=acct.pct_margin, hovertemplate="<br>Percent Margin: %{y:.1%}<br><extra></extra>")
fig.add_trace(trace)
# some formatting
fig.update_traces(marker_line_width=1, marker_line_color='black')
fig.layout.xaxis["title"] = "Time"
fig.layout.yaxis["title"] = "Percent Margin"
fig.add_hline(y=MAINTENANCE_MARGIN, line_width=4, line_dash="dash", line_color="black")
fig.add_annotation(x=3, y=MAINTENANCE_MARGIN*1.3,
            text="Maintenance margin: "+f'{MAINTENANCE_MARGIN:.1%}', showarrow=False)
fig.show()

### Incorporating interest income and lending fees

In [8]:
# Example with interest income and lending fees
HORIZON = 30  # in days
INTEREST_RATE = 0.02
LENDING_FEE = 0.005
# LENDING_FEE = 0.35      # Special lending fee

rebate_rate = INTEREST_RATE - LENDING_FEE

new_asset_value = INIT_VALUE*(1+RETURN)
new_short_proceeds = short_proceeds*(1+rebate_rate*HORIZON/365)
new_margin = margin*(1+INTEREST_RATE*HORIZON/365)
new_equity = new_short_proceeds + new_margin - new_asset_value
pct_margin = (new_equity)/new_asset_value
print(f'New percent margin is: {pct_margin: .1%}')

New percent margin is:  28.7%
